CS4001/4042 Assignment 1, Part B, Q4
---

Model degradation is a common issue faced when deploying machine learning models (including neural networks) in the real world. New data points could exhibit a different pattern from older data points due to factors such as changes in government policy or market sentiments. For instance, housing prices in Singapore have been increasing and the Singapore government has introduced 3 rounds of cooling measures over the past years (16 December 2021, 30 September 2022, 27 April 2023).

In such situations, the distribution of the new data points could differ from the original data distribution which the models were trained on. Recall that machine learning models often work with the assumption that the test distribution should be similar to train distribution. When this assumption is violated, model performance will be adversely impacted.  In the last part of this assignment, we will investigate to what extent model degradation has occurred.




---



---



Your co-investigators used a linear regression model to rapidly test out several combinations of train/test splits and shared with you their findings in a brief report attached in Appendix A below. You wish to investigate whether your deep learning model corroborates with their findings.

In [1]:
!pip install alibi-detect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 372.4/372.4 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 52.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 30.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 80.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 67.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 17.5 MB/s eta 0:00:00


In [2]:
!pip install pytorch_tabular[extra]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.5/122.5 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.8/81.8 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 829.5/829.5 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 38.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.5/887.5 MB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.0/190.0 kB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
SEED = 42

import os

import random
random.seed(SEED)

import numpy as np
np.random.seed(SEED)

import pandas as pd

from alibi_detect.cd import TabularDrift

from pytorch_tabular import TabularModel
from pytorch_tabular.models import CategoryEmbeddingModelConfig
from pytorch_tabular.config import (
    DataConfig,
    OptimizerConfig,
    TrainerConfig,
)

/usr/local/lib/python3.10/dist-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/usr/local/lib/python3.10/dist-packages/torchvision/image.so: undefined symbol: _ZN3c104cuda20CUDACachingAllocator9allocatorE'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


> Evaluate your model from B1 on data from year 2022 and report the test R2.

In [4]:
# TODO: Enter your code here
df_start = pd.read_csv('hdb_price_prediction.csv',index_col = None)
columns_to_drop = ['full_address', 'nearest_stn']
df = df_start.drop(columns_to_drop,axis=1)

continuous_columns = ["dist_to_nearest_stn", "dist_to_dhoby", "degree_centrality", "eigenvector_centrality", "remaining_lease_years", "floor_area_sqm"]
categorical_columns = ["month", "town", "flat_model_type", "storey_range"]

train_data = df[df['year'] <= 2019]
val_data = df[df['year'] == 2020]
test_data = df[df['year'] == 2022]

train_data.reset_index(drop=True, inplace=True)
val_data.reset_index(drop=True, inplace=True)
test_data.reset_index(drop=True, inplace=True)

drop_year = ['year']

train_data = train_data.drop(drop_year,axis=1)
val_data = val_data.drop(drop_year,axis=1)
test_data = test_data.drop(drop_year,axis=1)

In [5]:
# TODO: Enter your code here
data_config = DataConfig(
    target = ["resale_price"],
    continuous_cols = continuous_columns,
    categorical_cols = categorical_columns,
)

trainer_config = TrainerConfig(
    auto_lr_find = True,  # Runs the LRFinder to automatically derive a learning rate
    batch_size = 1024,
    max_epochs = 50,
)

model_config = CategoryEmbeddingModelConfig(
    task = "regression",
    layers = "50",  # Number of nodes in each layer
    activation = "ReLU",  # Activation between each layers
    learning_rate = 1e-3,
    metrics=["mean_squared_error"]
)

optimizer_config = OptimizerConfig()

tabular_model_1 = TabularModel(
    data_config = data_config,
    model_config = model_config,
    optimizer_config = optimizer_config,
    trainer_config = trainer_config,
)

#save_dataloader = tabular_model.prepare_dataloader(train_dataloader, val_dataloader, test_dataloader)
tabular_model_1.fit(train = train_data, validation = val_data)
result = tabular_model_1.evaluate(test_data)
pred_df = tabular_model_1.predict(test_data)
tabular_model_1.save_model("examples/basic")
loaded_model = TabularModel.load_from_checkpoint("examples/basic")

2023-10-13 07:32:41,739 - {pytorch_tabular.tabular_model:105} - INFO - Experiment Tracking is turned off
INFO:pytorch_tabular.tabular_model:Experiment Tracking is turned off
INFO:lightning_fabric.utilities.seed:Global seed set to 42
2023-10-13 07:32:41,807 - {pytorch_tabular.tabular_model:473} - INFO - Preparing the DataLoaders
INFO:pytorch_tabular.tabular_model:Preparing the DataLoaders
2023-10-13 07:32:41,814 - {pytorch_tabular.tabular_datamodule:290} - INFO - Setting up the datamodule for regression task
INFO:pytorch_tabular.tabular_datamodule:Setting up the datamodule for regression task
2023-10-13 07:32:42,183 - {pytorch_tabular.tabular_model:521} - INFO - Preparing the Model: CategoryEmbeddingModel
INFO:pytorch_tabular.tabular_model:Preparing the Model: CategoryEmbeddingModel
2023-10-13 07:32:42,258 - {pytorch_tabular.tabular_model:268} - INFO - Preparing the Trainer
INFO:pytorch_tabular.tabular_model:Preparing the Trainer
/usr/local/lib/python3.10/dist-packages/pytorch_lightning

Finding best initial lr:   0%|          | 0/100 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/core/module.py:493: UserWarning: You called `self.log('train_loss', ..., logger=True)` but have no logger configured. You can enable one by doing `Trainer(logger=ALogger(...))`
  rank_zero_warn(
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/core/module.py:493: UserWarning: You called `self.log('train_mean_squared_error', ..., logger=True)` but have no logger configured. You can enable one by doing `Trainer(logger=ALogger(...))`
  rank_zero_warn(
INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_steps=100` reached.
INFO:pytorch_lightning.tuner.lr_finder:Learning rate set to 0.5754399373371567
INFO:pytorch_lightning.utilities.rank_zero:Restoring states from the checkpoint path at /content/.lr_find_fc124c08-9237-4d0a-8a21-e7d2085fff61.ckpt
INFO:pytorch_lightning.utilities.rank_zero:Restored all states from the checkpoint file at /content/.lr_find_fc124c08-9237-4d0a-8a21-e7d2085fff61.ckpt
2023-10-13 

┏━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃   ┃ Name             ┃ Type                      ┃ Params ┃
┡━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│ 0 │ _backbone        │ CategoryEmbeddingBackbone │  2.9 K │
│ 1 │ _embedding_layer │ Embedding1dLayer          │  1.5 K │
│ 2 │ head             │ LinearHead                │     51 │
│ 3 │ loss             │ MSELoss                   │      0 │
└───┴──────────────────┴───────────────────────────┴────────┘

Trainable params: 4.5 K                                                                                            
Non-trainable params: 0                                                                                            
Total params: 4.5 K                                                                                                
Total estimated model params size (MB): 0

...

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/core/module.py:493: UserWarning: You called 
`self.log('test_mean_squared_error', ..., logger=True)` but have no logger configured. You can enable one by doing 
`Trainer(logger=ALogger(...))`
  rank_zero_warn(

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/utilities/cloud_io.py:33: LightningDeprecationWarning: `pytorch_lightning.utilities.cloud_io.get_filesystem` has been deprecated in v1.8.0 and will be removed in v2.0.0. Please use `lightning_fabric.utilities.cloud_io.get_filesystem` instead.
  rank_zero_deprecation(


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_loss         │       16267038720.0       │
│  test_mean_squared_error  │       16267038720.0       │
└───────────────────────────┴───────────────────────────┘

Output()

2023-10-13 07:33:53,185 - {pytorch_tabular.tabular_model:129} - INFO - Experiment Tracking is turned off
INFO:pytorch_tabular.tabular_model:Experiment Tracking is turned off
2023-10-13 07:33:53,192 - {pytorch_tabular.tabular_model:268} - INFO - Preparing the Trainer
INFO:pytorch_tabular.tabular_model:Preparing the Trainer
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:589: LightningDeprecationWarning: The Trainer argument `auto_select_gpus` has been deprecated in v1.9.0 and will be removed in v2.0.0. Please use the function `pytorch_lightning.accelerators.find_usable_cuda_devices` instead.
  rank_zero_deprecation(
INFO:pytorch_lightning.utilities.rank_zero:Trainer already configured with model summary callbacks: [<class 'pytorch_lightning.callbacks.rich_model_summary.RichModelSummary'>]. Skipping setting a default `ModelSummary` callback.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_ligh

In [6]:
result
rmse = np.sqrt(result[0]['test_mean_squared_error'])
print(rmse)

127542.30168849863


In [7]:
from sklearn.metrics import mean_squared_error, r2_score
target = ["resale_price"]
predict_target_column = ["resale_price_prediction"]
# Calculating RMSE
rmse = np.sqrt(mean_squared_error(test_data["resale_price"], pred_df["resale_price_prediction"]))

# Calculating R-squared (R2) value
r2 = r2_score(test_data[target], pred_df[predict_target_column])

print(f"Test RMSE: {rmse:.2f}")
print(f"Test R2: {r2:.2f}")


Test RMSE: 127542.30
Test R2: 0.44


In [8]:
# Creating a dataframe to show the results.
confirming_results = {'Training set': ["Year <= 2019"], 'Test set': ['2021'], 'Test R2': [0.78]} # Test R2 retrieved from model in B1
confirming_results_df = pd.DataFrame(data = confirming_results)

confirming_results_df.loc[len(confirming_results_df.index)] = ["Year <= 2019", '2022', r2]

> Evaluate your model from B1 on data from year 2023 and report the test R2.

In [9]:
# TODO: Enter your code here
test_data_part2 = df[df['year'] == 2023]

test_data_part2.reset_index(drop=True, inplace=True)

drop_year = ['year']


test_data_part2 = test_data_part2.drop(drop_year,axis=1)

In [10]:
result_part2 = tabular_model_1.evaluate(test_data_part2)
pred_df_part2 = tabular_model_1.predict(test_data_part2)

Output()

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/core/module.py:493: UserWarning: You called 
`self.log('test_loss', ..., logger=True)` but have no logger configured. You can enable one by doing 
`Trainer(logger=ALogger(...))`
  rank_zero_warn(

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/core/module.py:493: UserWarning: You called 
`self.log('test_mean_squared_error', ..., logger=True)` but have no logger configured. You can enable one by doing 
`Trainer(logger=ALogger(...))`
  rank_zero_warn(

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_loss         │       24701366272.0       │
│  test_mean_squared_error  │       24701366272.0       │
└───────────────────────────┴───────────────────────────┘

Output()

In [11]:
rmse = np.sqrt(result_part2[0]['test_mean_squared_error'])
print(rmse)

157166.68308518827


In [12]:
rmse = np.sqrt(mean_squared_error(test_data_part2[target], pred_df_part2[predict_target_column]))

# Calculating R-squared (R2) value
r2 = r2_score(test_data_part2[target], pred_df_part2[predict_target_column])

print(f"Test RMSE: {rmse:.2f}")
print(f"Test R2: {r2:.2f}")
confirming_results_df.loc[len(confirming_results_df.index)] = ["Year <= 2019", '2023', r2]

Test RMSE: 157166.68
Test R2: 0.16


In [13]:
confirming_results_df

,Training set,Test set,Test R2
0,Year <= 2019,2021,0.780000
1,Year <= 2019,2022,0.438847
2,Year <= 2019,2023,0.162126


> Did model degradation occur for the deep learning model?


\# TODO: \<Yes, the model has degraded. We see that even our analysis follows a similar pattern as the co-investigators. That is using a model that is trained on old data only and hasn't been trained on recent data deteriorates with time. The increase in the rmse and the decrease in R2 value also support our analysis.\>



---



---



Model degradation could be caused by [various data distribution shifts](https://huyenchip.com/2022/02/07/data-distribution-shifts-and-monitoring.html#data-shift-types): covariate shift (features), label shift and/or concept drift (altered relationship between features and labels).
There are various conflicting terminologies in the [literature](https://www.sciencedirect.com/science/article/pii/S0950705122002854#tbl1). Let’s stick to this reference for this assignment.

> Using the **Alibi Detect** library, apply the **TabularDrift** function with the training data (year 2019 and before) used as the reference and **detect which features have drifted** in the 2023 test dataset. Before running the statistical tests, ensure you **sample 1000 data points** each from the train and test data. Do not use the whole train/test data. (Hint: use this example as a guide https://docs.seldon.io/projects/alibi-detect/en/stable/examples/cd_chi2ks_adult.html)


In [14]:
# TODO: Enter your code here
target_remove = ["resale_price"]
X_ref = train_data.drop(target_remove,axis=1)
y_ref = train_data[target_remove]
X_test = test_data_part2.drop(target_remove,axis=1)
y_test = test_data_part2[target_remove]
#X_ref_sample  = X_ref.to_numpy() # For the entire datset, but not needed.
#y_ref_sample = y_ref.to_numpy()
#X_test_sample = X_test.to_numpy()
#y_test_sample = y_test.to_numpy()
X_ref_sample  = X_ref.sample(1000, random_state = SEED).to_numpy()
y_ref_sample = y_ref.sample(1000, random_state = SEED).to_numpy()
X_test_sample = X_test.sample(1000, random_state = SEED).to_numpy()
y_test_sample = y_test.sample(1000, random_state = SEED).to_numpy()

In [15]:
X_ref_sample

array([[12, 'JURONG EAST', 0.985652039, ..., 64.16666667, 67.0,
        '07 TO 09'],
       [4, 'BUKIT MERAH', 0.15449351, ..., 81.58333333, 121.0,
        '22 TO 24'],
       [12, 'PUNGGOL', 1.079198298, ..., 84.33333333, 128.0, '10 TO 12'],
       ...,
       [11, 'TAMPINES', 0.28728573, ..., 91.33333333, 110.0, '01 TO 03'],
       [1, 'SERANGOON', 1.797086041, ..., 67.16666667, 121.0, '04 TO 06'],
       [3, 'SENGKANG', 0.686580828, ..., 84.16666667, 110.0, '04 TO 06']],
      dtype=object)

In [16]:
# Creating the feature map.
categories_per_feature = {}
i = 0
for column_name in train_data.columns:
  if column_name in categorical_columns:
    categories_per_feature[i] = None
    #categories_per_feature[i] = X_ref[column_name].unique()
  i += 1

print(categories_per_feature)


{0: None, 1: None, 6: None, 9: None}


In [17]:
cd = TabularDrift(X_ref_sample, p_val=.05, categories_per_feature = categories_per_feature)

In [18]:
fpreds = cd.predict(X_test_sample, drift_type='feature') #Detecting drift at the festure level
fpreds

{'data': {'is_drift': array([1, 0, 0, 0, 0, 0, 1, 1, 1, 1]),
  'distance': array([4.3033649e+02, 3.3177677e+01, 3.5000000e-02, 5.9000000e-02,
         3.7999999e-02, 5.6000002e-02, 6.2121868e+01, 1.6300000e-01,
         6.1999999e-02, 2.7842325e+01], dtype=float32),
  'p_val': array([0.0000000e+00, 1.2668547e-01, 5.6057990e-01, 5.9107810e-02,
         4.5465949e-01, 8.3652966e-02, 1.1043324e-03, 4.4748402e-12,
         4.1015215e-02, 9.5192054e-03], dtype=float32),
  'threshold': 0.05},
 'meta': {'name': 'TabularDrift',
  'online': False,
  'data_type': None,
  'version': '0.11.4',
  'detector_type': 'drift'}}

In [20]:
drift_df_column_names = ["Feature Name", "Statistic Test", "Statistic Test Value", "Is Drift", "p-val"]
drift_df = pd.DataFrame(columns = drift_df_column_names)

In [21]:
labels = ['No!', 'Yes!']
# Seeing the raw values as output
for f in range(cd.n_features):
    stat = 'Chi2' if f in list(categories_per_feature.keys()) else 'K-S'
    fname = X_ref.columns[f]
    is_drift = fpreds['data']['is_drift'][f]
    drifted_or_not = labels[is_drift]
    stat_val, p_val = fpreds['data']['distance'][f], fpreds['data']['p_val'][f]
    data = {
        "Feature Name": fname,
        "Statistic Test": stat,
        "Statistic Test Value": stat_val,
        "Is Drift": drifted_or_not,
        "p-val": p_val,
    }
    drift_df = drift_df.append(data, ignore_index=True)
    print(f'{fname} -- Drift? {labels[is_drift]} -- {stat} {stat_val:.3f} -- p-value {p_val:.3f}')

month -- Drift? Yes! -- Chi2 430.336 -- p-value 0.000
town -- Drift? No! -- Chi2 33.178 -- p-value 0.127
dist_to_nearest_stn -- Drift? No! -- K-S 0.035 -- p-value 0.561
dist_to_dhoby -- Drift? No! -- K-S 0.059 -- p-value 0.059
degree_centrality -- Drift? No! -- K-S 0.038 -- p-value 0.455
eigenvector_centrality -- Drift? No! -- K-S 0.056 -- p-value 0.084
flat_model_type -- Drift? Yes! -- Chi2 62.122 -- p-value 0.001
remaining_lease_years -- Drift? Yes! -- K-S 0.163 -- p-value 0.000
floor_area_sqm -- Drift? Yes! -- K-S 0.062 -- p-value 0.041
storey_range -- Drift? Yes! -- Chi2 27.842 -- p-value 0.010


<ipython-input-21-57cea24bbfb8>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  drift_df = drift_df.append(data, ignore_index=True)
<ipython-input-21-57cea24bbfb8>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  drift_df = drift_df.append(data, ignore_index=True)
<ipython-input-21-57cea24bbfb8>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  drift_df = drift_df.append(data, ignore_index=True)
<ipython-input-21-57cea24bbfb8>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  drift_df = drift_df.append(data, ignore_index=True)
<ipython-input-21-57cea24bbfb8>:16: FutureWarning: The frame.append method is deprecated and will be removed

In [22]:
drift_df # Clearly states the results

,Feature Name,Statistic Test,Statistic Test Value,Is Drift,p-val
0,month,Chi2,430.336487,Yes!,0.000000e+00
1,town,Chi2,33.177677,No!,1.266855e-01
2,dist_to_nearest_stn,K-S,0.035000,No!,5.605799e-01
3,dist_to_dhoby,K-S,0.059000,No!,5.910781e-02
4,degree_centrality,K-S,0.038000,No!,4.546595e-01
5,eigenvector_centrality,K-S,0.056000,No!,8.365297e-02
6,flat_model_type,Chi2,62.121868,Yes!,1.104332e-03
7,remaining_lease_years,K-S,0.163000,Yes!,4.474840e-12
8,floor_area_sqm,K-S,0.062000,Yes!,4.101522e-02
9,storey_range,Chi2,27.842325,Yes!,9.519205e-03


> Assuming that the flurry of housing measures have made an impact on the relationship between all the features and resale_price (i.e. P(Y|X) changes), which type of data distribution shift possibly led to model degradation?


\# TODO: \<Concept Shift. It seems that the relationship between the input features and output resale_price has changed. It could perhaps be because of changes in demand and inflation after covid-19\>

> From your analysis via TabularDrift, which features contribute to this shift?


\# TODO: \<The features that contribute to the shift are: month, flat_model_type_remaining_lease_years, floor_area_sqm, storey_range\>

> Suggest 1 way to address model degradation and implement it, showing improved test R2 for year 2023.


\# TODO: \<One way to address concept drift is by regularly retraining our neural network using updated and recent data. In our situation we can train the model based on 2021 data to improve the test accuracy for 2023 data. As seen in the appendix, the closer the year feature of the training set is to the year in the test set, the higher the R2 scores are. This is because the difference in economic trends across consecutive years is gradual, hence resulting in better predictions when the model is trained on recent data.\>

In [23]:
# TODO: Enter your code here
train_data_improve = df[df['year'] == 2021]
val_data_improve = df[df['year'] == 2022]
test_data_improve= df[df['year'] == 2023]

train_data_improve.reset_index(drop=True, inplace=True)
val_data_improve.reset_index(drop=True, inplace=True)
test_data_improve.reset_index(drop=True, inplace=True)

drop_year = ['year']

train_data_improve = train_data_improve.drop(drop_year,axis=1)
val_data_improve = val_data_improve.drop(drop_year,axis=1)
test_data_improve = test_data_improve.drop(drop_year,axis=1)

In [24]:
data_config_improve = DataConfig(
    target = ["resale_price"],
    continuous_cols = continuous_columns,
    categorical_cols = categorical_columns,
)

trainer_config_improve = TrainerConfig(
    auto_lr_find = True,  # Runs the LRFinder to automatically derive a learning rate
    batch_size = 1024,
    max_epochs = 50,
)

model_config_improve = CategoryEmbeddingModelConfig(
    task = "regression",
    layers = "50",  # Number of nodes in each layer
    activation = "ReLU",  # Activation between each layers
    learning_rate = 1e-3,
    metrics=["mean_squared_error"]
)

optimizer_config_improve = OptimizerConfig()

tabular_model_improve = TabularModel(
    data_config = data_config_improve,
    model_config = model_config_improve,
    optimizer_config = optimizer_config_improve,
    trainer_config = trainer_config_improve,
)
tabular_model_improve.fit(train = train_data_improve, validation = val_data_improve)
result_improve = tabular_model_improve.evaluate(test_data_improve)
pred_df_improve = tabular_model_improve.predict(test_data_improve)

2023-10-13 07:40:17,974 - {pytorch_tabular.tabular_model:105} - INFO - Experiment Tracking is turned off
INFO:pytorch_tabular.tabular_model:Experiment Tracking is turned off
INFO:lightning_fabric.utilities.seed:Global seed set to 42
2023-10-13 07:40:18,008 - {pytorch_tabular.tabular_model:473} - INFO - Preparing the DataLoaders
INFO:pytorch_tabular.tabular_model:Preparing the DataLoaders
2023-10-13 07:40:18,014 - {pytorch_tabular.tabular_datamodule:290} - INFO - Setting up the datamodule for regression task
INFO:pytorch_tabular.tabular_datamodule:Setting up the datamodule for regression task
2023-10-13 07:40:18,173 - {pytorch_tabular.tabular_model:521} - INFO - Preparing the Model: CategoryEmbeddingModel
INFO:pytorch_tabular.tabular_model:Preparing the Model: CategoryEmbeddingModel
2023-10-13 07:40:18,215 - {pytorch_tabular.tabular_model:268} - INFO - Preparing the Trainer
INFO:pytorch_tabular.tabular_model:Preparing the Trainer
/usr/local/lib/python3.10/dist-packages/pytorch_lightning

Finding best initial lr:   0%|          | 0/100 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/core/module.py:493: UserWarning: You called `self.log('train_loss', ..., logger=True)` but have no logger configured. You can enable one by doing `Trainer(logger=ALogger(...))`
  rank_zero_warn(
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/core/module.py:493: UserWarning: You called `self.log('train_mean_squared_error', ..., logger=True)` but have no logger configured. You can enable one by doing `Trainer(logger=ALogger(...))`
  rank_zero_warn(
INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_steps=100` reached.
INFO:pytorch_lightning.tuner.lr_finder:Learning rate set to 0.8317637711026709
INFO:pytorch_lightning.utilities.rank_zero:Restoring states from the checkpoint path at /content/.lr_find_01c778bc-a2d4-466b-8775-fb44cfbd7239.ckpt
INFO:pytorch_lightning.utilities.rank_zero:Restored all states from the checkpoint file at /content/.lr_find_01c778bc-a2d4-466b-8775-fb44cfbd7239.ckpt
2023-10-13 

┏━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃   ┃ Name             ┃ Type                      ┃ Params ┃
┡━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│ 0 │ _backbone        │ CategoryEmbeddingBackbone │  3.0 K │
│ 1 │ _embedding_layer │ Embedding1dLayer          │  1.6 K │
│ 2 │ head             │ LinearHead                │     51 │
│ 3 │ loss             │ MSELoss                   │      0 │
└───┴──────────────────┴───────────────────────────┴────────┘

Trainable params: 4.6 K                                                                                            
Non-trainable params: 0                                                                                            
Total params: 4.6 K                                                                                                
Total estimated model params size (MB): 0

...

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/core/module.py:493: UserWarning: You called 
`self.log('test_loss', ..., logger=True)` but have no logger configured. You can enable one by doing 
`Trainer(logger=ALogger(...))`
  rank_zero_warn(

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/core/module.py:493: UserWarning: You called 
`self.log('test_mean_squared_error', ..., logger=True)` but have no logger configured. You can enable one by doing 
`Trainer(logger=ALogger(...))`
  rank_zero_warn(

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/utilities/cloud_io.py:33: LightningDeprecationWarning: `pytorch_lightning.utilities.cloud_io.get_filesystem` has been deprecated in v1.8.0 and will be removed in v2.0.0. Please use `lightning_fabric.utilities.cloud_io.get_filesystem` instead.
  rank_zero_deprecation(


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_loss         │       12147093504.0       │
│  test_mean_squared_error  │       12147093504.0       │
└───────────────────────────┴───────────────────────────┘

Output()

In [25]:
target = ["resale_price"]
predict_target_column = ["resale_price_prediction"]
# Calculating RMSE
rmse = np.sqrt(mean_squared_error(test_data_improve["resale_price"], pred_df_improve["resale_price_prediction"]))

# Calculating R-squared (R2) value
r2 = r2_score(test_data_improve[target], pred_df_improve[predict_target_column])

print(f"Test RMSE: {rmse:.2f}")
print(f"Test R2: {r2:.2f}")
print("Higher Accuracy than before:)")

Test RMSE: 110213.86
Test R2: 0.59
Higher Accuracy than before:)


### Appendix A



Here are our results from a linear regression model. We used StandardScaler for continuous variables and OneHotEncoder for categorical variables.

While 2021 data can be predicted well, test R2 dropped rapidly for 2022 and 2023.

| Training set | Test set | Test R2 |
|--------------|----------|---------|
| Year <= 2020 | 2021     | 0.76    |
| Year <= 2020 | **2022**     | 0.41    |
| Year <= 2020 | **2023**     | **0.10**   |



Similarly, a model trained on 2017 data can predict 2018-2021 well (with slight degradation in performance for 2021), but drops drastically in 2022 and 2023.

| Training set | Test set | Test R2 |
|--------------|----------|---------|
| 2017         | 2018     | 0.90    |
|              | 2019     | 0.89    |
|              | 2020     | 0.87    |
|              | 2021     | 0.72    |
|              | **2022**     | **0.37**    |
|              | **2023**     | **0.09**    |

With the test set fixed at year 2021, training on data from 2017-2020 still works well on the test data, with minimal degradation. Training sets closer to year 2021 generally do better.

| Training set | Test set | Test R2 |
|--------------|----------|---------|
| 2020         | 2021     | 0.81    |
| 2019         | 2021     | 0.75    |
| 2018         | 2021     | 0.73    |
| 2017         | 2021     | 0.72    |